#Making Squad

In [1]:
!nvidia-smi

Mon Dec 26 02:18:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers datasets
from transformers import AutoTokenizer
from transformers import AdamW, BertTokenizerFast, BertConfig, get_linear_schedule_with_warmup
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 33.0 MB/s 
     |████████████████████████████████| 452 kB 71.8 MB/s 
     |████████████████████████████████| 7.6 MB 53.6 MB/s 
     |████████████████████████████████| 182 kB 74.7 MB/s 
     |████████████████████████████████| 132 kB 61.5 MB/s 
     |████████████████████████████████| 212 kB 81.9 MB/s 
     |████████████████████████████████| 127 kB 78.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import json
import os
train_data = json.load(open('drive/MyDrive/ntu-adl-hw2-spring-2021/train.json'))
valid_data = json.load(open('drive/MyDrive/ntu-adl-hw2-spring-2021/valid.json'))
Context = json.load(open('drive/MyDrive/ntu-adl-hw2-spring-2021/context.json'))
print(train_data[0])

Mounted at /content/drive
{'id': '593f14f960d971e294af884f0194b3a7', 'question': '舍本和誰的數據能推算出連星的恆星的質量？', 'paragraphs': [2018, 6952, 8264, 836], 'relevant': 836, 'answer': {'text': '斯特魯維', 'start': 108}}


In [4]:
question = []
context = []
answer = []
for data in train_data:
    question.append(data['question'])
    context.append(Context[data['relevant']])
    answer.append({"answer_start": [data['answer']['start']], "text": [data['answer']['text']]})

train = dict({
    'question': question,
    'context': context,
    'answers': answer,
})
question = []
context = []
answer = []
for data in valid_data:
    question.append(data['question'])
    context.append(Context[data['relevant']])
    answer.append({"answer_start":[data['answer']['start']], "text": [data['answer']['text']]})

valid = dict({
    'question': question,
    'context': context,
    'answers': answer,
})
from datasets import Dataset
train_dataset = Dataset.from_dict(train)
valid_dataset = Dataset.from_dict(valid)
train_dataset
valid_dataset

Dataset({
    features: ['question', 'context', 'answers'],
    num_rows: 3009
})

In [5]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [6]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
examples = train_dataset[:5]
features = prepare_train_features(examples)
print(features['input_ids'][2])
idx = 2
[tokenizer.decode(features["input_ids"][idx])]

[101, 519, 3300, 7095, 1070, 8024, 1921, 678, 4261, 511, 4192, 7095, 2180, 8024, 1921, 678, 3926, 511, 520, 2900, 4638, 3221, 862, 782, 3122, 4788, 5984, 7377, 2527, 8024, 4372, 6725, 7688, 4192, 7095, 7095, 2255, 3229, 4638, 677, 3292, 136, 102, 519, 4192, 7095, 520, 671, 1399, 4638, 4507, 889, 2792, 6303, 7525, 1914, 8024, 1001, 3064, 3297, 2451, 4638, 6223, 7953, 3975, 5632, 7681, 1918, 7983, 4638, 517, 3346, 1453, 1154, 1751, 2562, 518, 8038, 2782, 1751, 3314, 2399, 8024, 4912, 4374, 2088, 3124, 1920, 2200, 4374, 5430, 3122, 3504, 8024, 4788, 5984, 7377, 2527, 8024, 4372, 6725, 7688, 4192, 7095, 7095, 2255, 511, 6725, 1894, 1813, 7102, 6863, 7613, 2533, 4767, 4811, 8024, 677, 3292, 519, 3300, 7095, 1070, 8024, 1921, 678, 4261, 511, 4192, 7095, 2180, 8024, 1921, 678, 3926, 511, 520, 5445, 3634, 1184, 8024, 4192, 7095, 4638, 7095, 2255, 4670, 4496, 7095, 4846, 8024, 4158, 1070, 2157, 2553, 4261, 722, 1765, 8024, 4374, 5430, 6876, 6241, 6210, 3634, 4811, 1921, 678, 1377, 1922, 2398, 5

['[CLS] 「 有 錫 兵 ， 天 下 爭 。 無 錫 寧 ， 天 下 清 。 」 指 的 是 何 人 攻 破 蘭 陵 後 ， 率 軍 駐 無 錫 錫 山 時 的 上 書? [SEP] 「 無 錫 」 一 名 的 由 來 所 說 頗 多 ， 傳 播 最 廣 的 觀 點 源 自 馮 夢 龍 的 《 東 周 列 國 志 》 ： 戰 國 末 年 ， 秦 王 嬴 政 大 將 王 翦 攻 楚 ， 破 蘭 陵 後 ， 率 軍 駐 無 錫 錫 山 。 軍 士 埋 鍋 造 飯 得 石 碑 ， 上 書 「 有 錫 兵 ， 天 下 爭 。 無 錫 寧 ， 天 下 清 。 」 而 此 前 ， 無 錫 的 錫 山 盛 產 錫 礦 ， 為 兵 家 必 爭 之 地 ， 王 翦 遂 言 見 此 碑 天 下 可 太 平 。 「 無 錫 」 的 地 名 便 這 樣 誕 生 了 ， 此 後 無 錫 歷 代 地 方 志 中 亦 據 此 訛 載 頗 多 。 根 據 近 幾 年 語 言 學 家 在 音 韻 學 方 面 的 研 究 ， 無 錫 的 地 名 可 能 來 自 古 越 族 的 稱 呼 。 「 無 」 是 發 語 詞 ， 沒 有 實 際 意 義 ， 「 錫 」 的 原 義 因 古 越 語 佚 亡 已 久 ， 無 從 考 證 ， 和 江 浙 地 區 的 姑 蘇 、 餘 杭 、 夫 椒 等 地 名 一 樣 ， 都 屬 於 齊 頭 式 地 名 ， 為 古 越 語 的 發 語 詞 ， 無 實 際 意 義 。 但 據 語 言 學 家 鄭 張 尚 芳 考 證 ， 「 無 」 為 古 吳 語 的 敬 語 前 綴 ， 「 無 錫 」 的 意 思 就 是 尊 敬 的 錫 山 山 神 。 鄭 張 尚 芳 對 「 姑 蘇 、 餘 杭 、 夫 椒 、 盱 眙 」 等 地 名 的 [SEP]']

In [ ]:
train_squad = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
valid_squad = valid_dataset.map(prepare_train_features, batched=True, remove_columns=valid_dataset.column_names)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

# Training

In [ ]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese")

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

In [ ]:
os.makedirs('/content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa', exist_ok=True)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_squad,
    eval_dataset=valid_squad,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 38143
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 47680


Epoch,Training Loss,Validation Loss
1,0.895900,0.703799
2,0.503900,0.865365
3,0.572200,1.449494
4,0.543300,1.599244
5,0.455000,1.771584


***** Running Evaluation *****
  Num examples = 5503
  Batch size = 2
Saving model checkpoint to /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-9536
Configuration saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-9536/config.json
Model weights saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-9536/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-9536/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-9536/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5503
  Batch size = 2
Saving model checkpoint to /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-19072
Configuration saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results_qa/checkpoint-19072/config.json
Model weights saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021

TrainOutput(global_step=47680, training_loss=0.5940753706349623, metrics={'train_runtime': 10630.6232, 'train_samples_per_second': 17.94, 'train_steps_per_second': 4.485, 'total_flos': 3.737490822067968e+16, 'train_loss': 0.5940753706349623, 'epoch': 5.0})